In [1]:
import json
import os

In [2]:
# Change these variables to perform evalution for you task
test_path = os.path.abspath(os.getcwd()).split('gpt3_completion_scripts')[0] + 'processed_data/gpt3/completion_4/processed_test.json'
generated_path = os.path.abspath(os.getcwd()).split('gpt3_completion_scripts')[0] + 'generated_data_gpt3/experiment_4/generated_distractors.json'
input_macaw_path = os.path.abspath(os.getcwd()).split('gpt3_completion_scripts')[0] + 'generated_data_gpt3/experiment_4/input_macaw.json'

In [3]:
# Get all the target prompts and completions
test_data = []

for line in open(test_path):
    test_data.append((json.loads(line)))
    
generated_data = []
# Get the generated completions
with open(generated_path) as f:
    generated = json.load(f)

for key in generated:
    generated_data.append(generated[key])

In [4]:
# Put the predictions and gold references in lists and calculate exact match and f1 score

predictions = []
gold_references = []

for i in range(len(test_data)):
    
    completion = test_data[i]['completion']
    clean_completion = completion.split('False answer: ')
    distractor_1 = clean_completion[1].split('\n')[0]
    distractor_2 = clean_completion[2].split('\n')[0]
    distractor_3 = clean_completion[3].split('\n')[0]
    
    gold_references.append([distractor_1, distractor_2, distractor_3])
    
    generated_completion = generated_data[i]
    clean_generated_completion = generated_completion.split('False answer:')
    generated_distractor_1 = clean_generated_completion[1].split('\n')[0].strip()
    generated_distractor_2 = clean_generated_completion[2].split('\n')[0].strip()
    generated_distractor_3 = clean_generated_completion[3].split('\n')[0].strip()

    predictions.append([generated_distractor_1, generated_distractor_2, generated_distractor_3])

In [5]:
# Create result files
with open(os.path.abspath(os.getcwd()).split('gpt3_completion_scripts')[0] + 'generated_data_gpt3/experiment_4/' + 'predictions_distractors.txt', 'w') as f:
    for i in range(len(predictions)):
        f.write(predictions[i][0] + '\n')
        f.write(predictions[i][1] + '\n')
        f.write(predictions[i][2] + '\n\n')
        
with open(os.path.abspath(os.getcwd()).split('gpt3_completion_scripts')[0] + 'generated_data_gpt3/experiment_4/' + 'ground_truth_distractors.txt', 'w') as f:
    for i in range(len(gold_references)):
        f.write(gold_references[i][0] + '\n')
        f.write(gold_references[i][1] + '\n')
        f.write(gold_references[i][2] + '\n\n')

# Create input file for testing accuracy with Macaw-11b

In [7]:
macaw_data_list = []
    
for i in range(len(test_data)):
    temp = test_data[i]['prompt']
    text = temp.split('\n\nQuestion: ')[0]
    question = temp.split('\n\nQuestion: ')[1].split('\n\nAnswer:')[0]
    answer = temp.split('\n\nAnswer: ')[1].split('\n\n###')[0]
    option_a = answer
    option_b = predictions[i][0]
    option_c = predictions[i][1]
    option_d = predictions[i][2]
    
    macaw_prompt_text = "$answer$" + " ; " + "$question$ = " + question + " $mcoptions$ = (A) " + option_a + \
                        " (B) " + option_b + " (C) " + option_c + " (D) " + option_d + " $context$ = " + text    
    macaw_completion = answer
    
    macaw_dict = {"prompt": macaw_prompt_text, "completion": macaw_completion}
    macaw_data_list.append(macaw_dict) 

In [8]:
with open(input_macaw_path, 'w') as f:
    for item in macaw_data_list:
        f.write(json.dumps(item) + "\n")  